# Imports in Python

_A quick workshop on the features and pitfalls of Python modules_

## Absolute imports

In [1]:
import module

In [2]:
module.VARIABLE

42

In [3]:
module.function()

__name__ module
42


In [4]:
!cat module.py


VARIABLE = 42

def function():
    print('__name__', __name__)
    print(VARIABLE)


### How did we choose this file?

Ubuntu Python:

```python
>>> import sys
>>> sys.path
['',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/home/krab/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages']
```

1. Working directory
2. Builtins
3. Packages installed to `~/.local/lib` (`pip3 install --user xyz`)
4. Extra packages installed to `/usr/local/lib` (`sudo pip3 install xyz`)
5. Packages installed to the distribution (`apt install python3-xyz`)

- Virtualenv changes the import locations
- IPython (in Jupyter) adds extra entries

In [5]:
import sys

sys.path

['',
 '/home/krab/.virtualenvs/pandas/lib/python36.zip',
 '/home/krab/.virtualenvs/pandas/lib/python3.6',
 '/home/krab/.virtualenvs/pandas/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '/home/krab/.virtualenvs/pandas/local/lib/python3.6/site-packages',
 '/home/krab/.virtualenvs/pandas/lib/python3.6/site-packages',
 '/home/krab/.virtualenvs/pandas/local/lib/python3.6/site-packages/IPython/extensions',
 '/home/krab/.ipython']

### You can see where the module comes from

In [6]:
module.__file__

'/home/krab/workspace/sedlakovi/python-imports/module.py'

### Most of the time...

In [7]:
!cat time.py

import time

time.sleep(1)
print('Good morning')


In [8]:
import time
time.__file__

AttributeError: module 'time' has no attribute '__file__'

In [9]:
time.__loader__

_frozen_importlib.BuiltinImporter

### It's a built-in module!

But current directory is searched first!

```python
>>> sys.meta_path
[<class '_frozen_importlib.BuiltinImporter'>,
 <class '_frozen_importlib.FrozenImporter'>,
 <class '_frozen_importlib_external.PathFinder'>]
```

### Finders and Loaders (since 3.3)

`meta_path` - list of finders (in order)

Finder finds a loader which can actually load the module.

### Finders and Loaders

- Can load package from 
  - database, 
  - network, 
  - .exe file, 
  - generate code on demand...

### Another gotcha!

In [10]:
!cat pandas.py

import pandas as pd

print(pd.read_csv('/tmp/test.csv'))


In [11]:
!python3 pandas.py

Traceback (most recent call last):
  File "pandas.py", line 1, in <module>
    import pandas as pd
  File "/home/krab/workspace/sedlakovi/python-imports/pandas.py", line 3, in <module>
    print(pd.read_csv('/tmp/test.csv'))
AttributeError: module 'pandas' has no attribute 'read_csv'


In [12]:
import pandas

AttributeError: module 'pandas' has no attribute 'read_csv'

### Current directory shadows installed packages

### Specialty - extending path

[`.pth` files](https://docs.python.org/3/library/site.html)

- Only works in `site-packages`
- Useful to "merge" two virtualenvs, e. g.
    1. base packages
    2. testing packages

## Relative imports

In [16]:
!rm -rf **/__pycache__ __pycache__
!tree -I presentations

.
├── Imports.ipynb
├── init_example.py
├── module.py
├── nested
│   ├── module.py
│   ├── neighbor.py
│   └── write.py
├── pandas.py
├── relative_test.py
└── time.py

1 directory, 9 files


In [17]:
!cat nested/neighbor.py

from . import module

print('Neigbor\'s neighbor lives at number:', module.VARIABLE)
print('Neigbor\'s neighbor is:', module.__file__)


In [18]:
import nested.neighbor

### Only from a package

In [19]:
!python3 -c 'from . import module'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ImportError: cannot import name 'module'


In [20]:
!cat relative_test.py
!echo
!echo '~~~~~~~~~~~~~~~~~~'
!echo
!python3 relative_test.py

from . import module

~~~~~~~~~~~~~~~~~~

Traceback (most recent call last):
  File "relative_test.py", line 1, in <module>
    from . import module
ImportError: cannot import name 'module'


In [21]:
import relative_test

ImportError: attempted relative import with no known parent package

In [22]:
!python3 nested/neighbor.py

Traceback (most recent call last):
  File "nested/neighbor.py", line 1, in <module>
    from . import module
ImportError: cannot import name 'module'


In [23]:
!python3 -m nested.neighbor

Neigbor's neighbor lives at number: 1337
Neigbor's neighbor is: /home/krab/workspace/sedlakovi/python-imports/nested/module.py


### Variants

(Formatting just for illustration here, don't add extra whitespace)

```python
from .          import   module
from .module    import   member, another_member as x, yet_another
from ..         import   module_in_parent_dir
from ...        import   module_in_parent_parent_dir
```

## Namespaces

In [24]:
!cat nested/write.py

from . import module


def update_neighbor():
    module.VARIABLE += 1
    print('My neighbor lives now at', module.VARIABLE)


In [25]:
import nested.write
import nested.module

nested.module.function()

Hi, I'm nested.module My VARIABLE is 1337


In [26]:
nested.write.update_neighbor()

My neighbor lives now at 1338


The module can write changes and see the value changed.

In [27]:
nested.write.update_neighbor()

My neighbor lives now at 1339


In [28]:
nested.module.function()

Hi, I'm nested.module My VARIABLE is 1339


The affected module can see the value.

In [29]:
nested.module.VARIABLE

1339

**Current module** can see the value changed - the namespace of `nested.module` is just one.

In fact, it lives in a global dict:

In [30]:
import sys
sys.modules['nested.module']

<module 'nested.module' from '/home/krab/workspace/sedlakovi/python-imports/nested/module.py'>

## Reloads

In [31]:
!cat init_example.py

print('Initializing module', __name__)


def func1():
    print('I can call', func2)


def func2():
    print('I can call', func1)


if __name__ == '__main__':
    print('I\'m the main!')
    func1()
    func2()


In [32]:
import init_example

Initializing module init_example


In [33]:
import init_example

In [34]:
import importlib

importlib.reload(init_example)
importlib.reload(init_example)
importlib.reload(init_example)
importlib.reload(init_example)

Initializing module init_example
Initializing module init_example
Initializing module init_example
Initializing module init_example


<module 'init_example' from '/home/krab/workspace/sedlakovi/python-imports/init_example.py'>

In [35]:
init_example.func1()

I can call <function func2 at 0x7fdbc5f408c8>


In [36]:
!python3 init_example.py

Initializing module __main__
I'm the main!
I can call <function func2 at 0x7f2a92b36598>
I can call <function func1 at 0x7f2a92b61e18>


In [37]:
init_example.__name__

'init_example'

## Dynamic imports

In [38]:
importlib.import_module('nested.mod' + 'ule')

<module 'nested.module' from '/home/krab/workspace/sedlakovi/python-imports/nested/module.py'>